In [6]:
# Imports
import pathlib
import os
from dataclasses import dataclass, field
from typing import Dict, Literal, Optional, Tuple
import json

import imutils
import numpy as np
import cv2
from dataclasses_json import DataClassJsonMixin
import pandas as pd
from tqdm import tqdm

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent
DATA_DIR = GIT_ROOT / "data" / 'PhotosynthesisFall2022'

In [7]:
# Game state reconstruction routines
@dataclass
class Participant(DataClassJsonMixin):
    id: str
    position: Tuple[float, float]
    state: Literal['null', 'H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O'] = 'null'

@dataclass
class EnvironmentState(DataClassJsonMixin):
    sun_state: Optional[bool] = None

@dataclass
class GameState(DataClassJsonMixin):
    participants: Dict[str, Participant] = field(default_factory=dict)
    environment: EnvironmentState = field(default_factory=EnvironmentState)

In [8]:
# Helper functions
def xy_transforms(xy: Tuple[float, float], w: int, h:int, c: Dict[str, float]) -> Tuple[float, float]:
    x, y = xy
    xx = int(((x+1)*h/2)*c['AFFINE'][0] + c['OFFSET'][0])
    yy = int(((y+1)*w/2)*c['AFFINE'][1] + c['OFFSET'][1])
    return (xx, yy)

def render(game_state: GameState, frame: np.ndarray, c: Dict[str, float]):

    if game_state.participants:
        for p in game_state.participants.values():
            p.position = (eval(p.position[0]), eval(p.position[1]))
            xy = xy_transforms(p.position, frame.shape[0], frame.shape[1], c)
            frame = cv2.circle(frame, xy, 30, (0, 0, 255), 1)
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (0,0,0), 
                2, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (255,255,255), 
                1, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (0,0,0), 
                2,
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (255,255,255), 
                1,
                cv2.LINE_AA
            )

    return frame

In [10]:
# Transformation constants
START_INDEX = 1000
CORRECTIONS = {'OFFSET': (-252,-280), 'AFFINE': (1.8,2)}
N = 20_000

# Load the data
vid_file = DATA_DIR / 'videos' / 'day 13' / "day13-screen-recording-corrected.mp4"
assert vid_file.exists()
cap = cv2.VideoCapture(str(vid_file))
fps = cap.get(cv2.CAP_PROP_FPS)

game_state_file = DATA_DIR / 'time_alignment' / 'day13_aligned_game_state.csv'
assert game_state_file.exists()
game_state_logs = pd.read_csv(game_state_file)

# Reset video and logs
cap.set(cv2.CAP_PROP_POS_FRAMES, START_INDEX)
game_state_pointer = 0

for i in tqdm(range(N), total=N):
    
    ret, frame = cap.read()
    if not ret:
        break

    frame_timestamp = (START_INDEX + i) / fps
    frame = cv2.putText(
        frame, 
        f"{pd.Timestamp(frame_timestamp, unit='s').strftime('%H:%M:%S.%f')}", 
        (5,30), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1, 
        (0,0,255), 
        1, 
        cv2.LINE_AA
    )

    while game_state_logs['timestamp'].iloc[game_state_pointer+1] < frame_timestamp:
        game_state_pointer += 1
    game_state = GameState.from_json(game_state_logs.iloc[game_state_pointer].state)

    # For macro alignment
    frame = cv2.putText(
        frame, 
        f"{game_state_pointer}", 
        (5,60), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1, 
        (0,0,255), 
        1, 
        cv2.LINE_AA
    )

    frame = render(game_state, frame, CORRECTIONS)
    cv2.imshow('frame', imutils.resize(frame, width=1000))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

  5%|▍         | 936/20000 [00:12<04:06, 77.47it/s]
